In [1]:
from sift1m_dataset_builder import Builder
import tensorflow_datasets.public_api as tfds
from etils.epath import Path

c:\Users\John\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = Path("B:/random ML datasets/dataset")
download_manager = tfds.download.DownloadManager(download_dir=path)

In [3]:
b = Builder()
split = b._split_generators(download_manager)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:01<00:00,  1.76s/ url]
Extraction completed...: 0 file [00:01, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:01<00:00,  1.76s/ url]


In [4]:
split.items()

dict_items([('database', <generator object Builder._generate_examples at 0x000002913C9ECEB0>), ('test', <generator object Builder._generate_examples at 0x000002913C9ED000>)])